# RAG Exercise Solution

In [1]:
import dotenv

from agents import Agent, Runner, trace, function_tool

import chromadb

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [2]:
# We populated the RAG with the data from the data/calories.csv file in the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient(path="../chroma")
calories_db = chroma_client.get_collection(name="nutrition_db")
nuntrition_qna_db = chroma_client.get_collection(name="nutrition_qna")

In [3]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too look up calorie information.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = calories_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)


@function_tool
def nutrtition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too ask a question about nutrition.

    Args:
        query: The question to ask
        max_results: The maximum number of results to return.

    Returns:
        A string containing the question and the answer related to the query.
    """

    results = nuntrition_qna_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Related answers to your question:\n" + "\n".join(formatted_results)

In [4]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information and nutrtion advice    .
    You give concise answers.
    
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrtition_qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool, nutrtition_qna_tool],
)

In [5]:
with trace("Nutrition Assistant with Nutrition and Calorie RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Pregnant women should focus on a balanced, nutrient-dense pattern with enough iron, folate, calcium, and DHA, plus an extra ~200 kcal/day. Aim for a mix of protein, whole grains, dairy or fortified alternatives, fruits/vegetables, and healthy fats.

Example day (about 2,200–2,400 kcal):
- Breakfast (~420 kcal): Greek yogurt with berries, a handful of nuts, and a drizzle of honey.
- Lunch (~700 kcal): Quinoa bowl with grilled chicken or beans, mixed greens, roasted veggies, avocado, and olive oil lemon dressing.
- Snack (~250 kcal): Apple or banana with peanut butter or a small cheese stick.
- Dinner (~700 kcal): Baked salmon or tofu, brown rice, steamed broccoli, and a side salad with vinaigrette.
- Evening snack (~150–300 kcal): Whole-grain crackers with hummus or a small milk-based smoothie.

Key nutrients to emphasize:
- Iron: lean meats, fortified cereals, beans, lentils, spinach (pair with vitamin C-rich foods to boost absorption).
- Folate: leafy greens, citrus, beans, fortified 